<a href="https://colab.research.google.com/github/ze-16/Conversational-AI-Agent/blob/main/Conversational_speech_training_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import gdown
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import BertModel
from torch.optim import AdamW
import time


In [2]:
# Coding the file path and allowing access through link
file_id = '1mC-mj6zuojm8jWRBH46Vj2Gno1ZsXlRF'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Text_data.csv'

In [3]:
# If statement to make sure file is only downloaded once and not repeatedly
if not os.path.exists(output):
    print("Downloading file...")
    gdown.download(url, output, quiet=False)
else:
    print(f"File already exists.")

Downloading...
From: https://drive.google.com/uc?id=1mC-mj6zuojm8jWRBH46Vj2Gno1ZsXlRF
To: /content/Text_data.csv
100%|██████████| 14.2M/14.2M [00:00<00:00, 36.8MB/s]


In [4]:
# Loading the goemotions_2 dataset for EDA
df = pd.read_csv(output)

In [5]:
# Loading the dimensions of the dataset
df.shape

(70000, 37)

In [6]:
# Loading the first 10 values of the dataset to explore what the data looks like
df.head(10)

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,We can hope,ee3o3ko,darkenseyreth,EdmontonOilers,t3_ag4r9j,t1_ee3mhad,1.547529e+09,62,False,0,...,0,0,1,0,0,0,0,0,0,0
1,Shhh don't give them the idea!,eebl3z7,BoinkBoinkEtAliae,MurderedByWords,t3_ah3o76,t1_eeb68lo,1.547777e+09,76,False,0,...,0,0,0,0,0,0,0,0,0,0
2,"Thank you so much, kind stranger. I really nee...",ed4fe9l,savageleaf,raisedbynarcissists,t3_abwh00,t1_ed4etbj,1.546482e+09,24,False,0,...,0,0,0,0,0,0,0,0,0,0
3,Ion know but it would be better for you to jus...,efavtdu,CADDiLLXC,darknet,t3_al4njw,t3_al4njw,1.548800e+09,62,False,0,...,0,0,0,0,0,0,0,0,0,1
4,I'm honestly surprised. We should have fallen ...,ee2imz2,CorporalThornberry,CollegeBasketball,t3_afxt6t,t1_ee22nyr,1.547497e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0
5,Jurisprudence fetishist gets off on technicality.,ednv5fy,100Gold,TopMindsOfReddit,t3_ae3p1m,t1_ednn3pz,1.547068e+09,12,False,0,...,0,0,0,0,0,0,0,0,0,1
6,One of my worst nightmares. Ugh.,ed60o6e,June1111,ExpectationVsReality,t3_ac7y4f,t3_ac7y4f,1.546541e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0
7,Is this real? 🤔,edjldcm,Uneekusername101,instant_regret,t3_adr232,t3_adr232,1.546928e+09,72,False,0,...,0,0,0,0,0,0,0,0,0,0
8,My favorite pod when they are fresh & clear. S...,ee7j34k,_BAMBOOZLE_,juul,t3_agn37x,t3_agn37x,1.547662e+09,12,False,0,...,0,0,0,0,0,0,0,0,0,1
9,I get wound up over completely imagined conver...,eeztbjr,TheLiquidSquid,aspergers,t3_ajpzf0,t1_eeypbzh,1.548483e+09,61,False,0,...,0,0,1,0,1,0,0,0,0,0


In [7]:
# Check the names of the headings and explore their data-types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   text                  70000 non-null  object 
 1   id                    70000 non-null  object 
 2   author                70000 non-null  object 
 3   subreddit             70000 non-null  object 
 4   link_id               70000 non-null  object 
 5   parent_id             70000 non-null  object 
 6   created_utc           70000 non-null  float64
 7   rater_id              70000 non-null  int64  
 8   example_very_unclear  70000 non-null  bool   
 9   admiration            70000 non-null  int64  
 10  amusement             70000 non-null  int64  
 11  anger                 70000 non-null  int64  
 12  annoyance             70000 non-null  int64  
 13  approval              70000 non-null  int64  
 14  caring                70000 non-null  int64  
 15  confusion          

In [8]:
# Basic descriptive statistics for categorical variables
df.describe(include='object')

,text,id,author,subreddit,link_id,parent_id
count,70000,70000,70000,70000,70000,70000
unique,43293,43448,37837,483,34995,41480
top,Thank you.,eep131n,[deleted],cringe,t3_ae6ejj,t3_ae6ejj
freq,18,5,1236,295,35,25


In [9]:
# Checking for any missing values and their sum for each column
df.isna().sum()

,0
text,0
id,0
author,0
subreddit,0
link_id,0
parent_id,0
created_utc,0
rater_id,0
example_very_unclear,0
admiration,0


In [10]:
# Removing variables/columns that will be of no consequence to the result upon initial look
df = df.drop(columns=(['id','author','subreddit','link_id','parent_id','created_utc','rater_id','example_very_unclear']))

In [11]:
# Running a check to make sure the targetted columns have been removed
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            70000 non-null  object
 1   admiration      70000 non-null  int64 
 2   amusement       70000 non-null  int64 
 3   anger           70000 non-null  int64 
 4   annoyance       70000 non-null  int64 
 5   approval        70000 non-null  int64 
 6   caring          70000 non-null  int64 
 7   confusion       70000 non-null  int64 
 8   curiosity       70000 non-null  int64 
 9   desire          70000 non-null  int64 
 10  disappointment  70000 non-null  int64 
 11  disapproval     70000 non-null  int64 
 12  disgust         70000 non-null  int64 
 13  embarrassment   70000 non-null  int64 
 14  excitement      70000 non-null  int64 
 15  fear            70000 non-null  int64 
 16  gratitude       70000 non-null  int64 
 17  grief           70000 non-null  int64 
 18  joy   

In [12]:
# Upon further reflection, dictionary created to condense the range of emotions, for the sake of simplicity of providing feedback at project end
emotion_groups = {
    'anger': ['anger', 'annoyance', 'disapproval', 'disgust', 'fear', 'nervousness'],
    'joy': ['joy', 'admiration', 'amusement', 'approval', 'caring', 'desire',
            'excitement', 'gratitude', 'love', 'optimism', 'pride', 'relief'],
    'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
    'neutral': ['neutral', 'confusion', 'curiosity', 'realization', 'surprise']
}

In [13]:
# Function that helps map the 28 emotions into 4 core groups, in order of priority
def get_core_emotion(row):
  for col in emotion_groups['anger']:
        if row[col] == 1:
            return 'anger'

  for col in emotion_groups['sadness']:
        if row[col] == 1:
            return 'sadness'

  for col in emotion_groups['joy']:
        if row[col] == 1:
            return 'joy'

  return 'neutral'

In [14]:
# Applying the above function onto the dataset
df['core_emotion'] = df.apply(get_core_emotion, axis=1)

In [15]:
# Checking to see the distribution of each category
df['core_emotion'].value_counts()

,count
core_emotion,
neutral,26126
joy,25520
anger,12900
sadness,5454


In [16]:
# Writing the new organised emotions into a new csv file
df[['text', 'core_emotion']].to_csv('Text_data2.csv', index=False)

In [17]:
# Loading the new file and checking if the mapping worked
df = pd.read_csv('Text_data2.csv')
df.head()

,text,core_emotion
0,We can hope,joy
1,Shhh don't give them the idea!,anger
2,"Thank you so much, kind stranger. I really nee...",joy
3,Ion know but it would be better for you to jus...,neutral
4,I'm honestly surprised. We should have fallen ...,joy


In [18]:
# Checking to see the distribution of each category
df['core_emotion'].value_counts()

,count
core_emotion,
neutral,26126
joy,25520
anger,12900
sadness,5454


In [19]:
# Loading a pre-trained bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
# Creating a class to tokenize dataset for BERT
class TokenizetoBERT(Dataset):
  # Storing the dataset and the tokenizer into the class
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
  # Gets the size of the data
    def __len__(self):
      return len(self.data)
  # Getting the text and labels and tokenizing them
    def __getitem__(self, index):
      # Getting the text
      text = str(self.data.iloc[index]['text'])
      # Getting the labels for the text
      labels = self.data.iloc[index]['core_emotion']
      # Mapping the strings to integers for computer interpretability
      label_map = {'neutral': 0, 'joy': 1, 'sadness': 2, 'anger': 3}
      label = label_map.get(labels, 0)
      # Tokenizing
      tokenizing = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

      return {
            'text': text,
            'input_ids': tokenizing['input_ids'].flatten(),
            'attention_mask': tokenizing['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [21]:
# Splitting the data based on variable 'core_emotion'
train_set,test_set = train_test_split(df,test_size=0.3, random_state=14,stratify=df['core_emotion'])

In [22]:
# Creating instances of the previously defined class
train_dataset = TokenizetoBERT(train_set, tokenizer)
test_dataset = TokenizetoBERT(test_set, tokenizer)

In [23]:
# Using the built in DataLoader class from pytorch
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=2,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=16,shuffle=True,num_workers=2,pin_memory=True)

In [24]:
# Initialising BERT in order to check for configurations
bert = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [25]:
# Checking for configurations for BERT
config = bert.config
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [26]:
# Deleting BERT and config and cleaning up memory
del bert
del config
torch.cuda.empty_cache()

In [27]:
# Configuring BERT model to my choice using a python class
class BertClassifier(nn.Module):
  def __init__(self, n_classes):
    super(BertClassifier, self).__init__()
    # Initialising BERT
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    # Changing the dropout rate, chance for any vector to be randomly set to 0
    self.drop = nn.Dropout(p=0.3)
    # Reducing the number of vectors from 768 to 32 for less noise and also reducing dimensionality for vector multiplication for fusion layer later on
    self.hl_reduction = nn.Linear (768,32)
    # Defining the final classifier
    self.classifier = nn.Linear (32,n_classes)

  def forward(self, input_ids, attention_mask):
        # Feeding tokenized text into BERT
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Getting the pooled output
        pooled_output = outputs.pooler_output
        output = self.drop(pooled_output)
        # Reducing the dimensions of the vectors
        reduced = self.hl_reduction(output)
        # Making the final prediction
        return self.classifier(reduced)

In [28]:
# Checking whether colab is utilising CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Creating instance of previous defined class/initialising the model and feeding n_classes=4 as a parameter
model = BertClassifier(n_classes=4)

# Transfers the model to "device"/GPU
model = model.to(device)

# Compiling the model for speed
model = torch.compile(model)

Using device: cuda


In [29]:
# Setting the optimizer with learning rate 2x10^-5
optimizer = AdamW(model.parameters(), lr=2e-5)

# Setting the weights of each class depending on the ratios between classes
weights = torch.tensor([1.0, 1.0, 5.0, 2.0]).to(device)

# Pre defined function calculating the loss using the weights defined above
loss_fn = nn.CrossEntropyLoss(weight=weights)

In [30]:
# Defining the number of epochs (training iterations) and variable to store the accuracy for each epoch
EPOCHS = 3
best_accuracy = 0

In [32]:
# Training loop
for epoch in range(EPOCHS):
    start_time = time.time()

    model.train()
    total_loss = 0
# Looping with two variables using enumerate
    for batch_num, batch in enumerate(train_loader):
        # Moving the data to the GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

# Training cycle
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
# Calculating the total loss using the pre defined .item function.
        total_loss += loss.item()

# If statement to make sure batch details are printed only on every 500th batch number
        if batch_num % 500 == 0:
            print(f"  Epoch {epoch+1} | Batch {batch_num}/{len(train_loader)} | Loss: {loss.item():.4f}")

# Calculating average training loss
    avg_train_loss = total_loss / len(train_loader)


    model.eval()
    test_accuracy = []

# Testing loop
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

# Making prediction from outputs and calculating the accuracy of the batch
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)
            acc = torch.sum(preds == labels).item() / len(preds)
            test_accuracy.append(acc)

# Calculating the mean of all of the batch prediction accuracies
    final_test_acc = np.mean(test_accuracy)

# Calculating time and printing the details of the epoch
    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1} Complete ({elapsed:.0f}s) | Train Loss: {avg_train_loss:.4f} | Test Acc: {final_test_acc:.4f}")

# If statement to save the best model
    if final_test_acc > best_accuracy:
        print(f"  Best model found")
        torch.save(model.state_dict(), 'best_bert_model.pth')
        best_accuracy = final_test_acc

print("Training Complete")

  Epoch 1 | Batch 0/3063 | Loss: 1.3716
  Epoch 1 | Batch 500/3063 | Loss: 1.1601
  Epoch 1 | Batch 1000/3063 | Loss: 0.9242
  Epoch 1 | Batch 1500/3063 | Loss: 1.0973
  Epoch 1 | Batch 2000/3063 | Loss: 1.4737
  Epoch 1 | Batch 2500/3063 | Loss: 1.0646
  Epoch 1 | Batch 3000/3063 | Loss: 1.0084


W1225 19:39:33.942000 5321 torch/fx/experimental/symbolic_shapes.py:6833] [0/1] _maybe_guard_rel() was called on non-relation expression Eq(s43, 1) | Eq(s72, s43)
W1225 19:40:21.834000 5321 torch/fx/experimental/symbolic_shapes.py:6833] [0/2] _maybe_guard_rel() was called on non-relation expression Eq(s43, 1) | Eq(s72, s43)


Epoch 1 Complete (640s) | Train Loss: 1.0150 | Test Acc: 0.5749
  Best model found
  Epoch 2 | Batch 0/3063 | Loss: 0.6364
  Epoch 2 | Batch 500/3063 | Loss: 0.7162
  Epoch 2 | Batch 1000/3063 | Loss: 1.3660
  Epoch 2 | Batch 1500/3063 | Loss: 0.8906
  Epoch 2 | Batch 2000/3063 | Loss: 1.2734
  Epoch 2 | Batch 2500/3063 | Loss: 0.8544
  Epoch 2 | Batch 3000/3063 | Loss: 0.9067
Epoch 2 Complete (578s) | Train Loss: 0.8811 | Test Acc: 0.5990
  Best model found
  Epoch 3 | Batch 0/3063 | Loss: 0.5396
  Epoch 3 | Batch 500/3063 | Loss: 0.4182
  Epoch 3 | Batch 1000/3063 | Loss: 0.7881
  Epoch 3 | Batch 1500/3063 | Loss: 0.5246
  Epoch 3 | Batch 2000/3063 | Loss: 0.9534
  Epoch 3 | Batch 2500/3063 | Loss: 0.8125
  Epoch 3 | Batch 3000/3063 | Loss: 0.9295
Epoch 3 Complete (578s) | Train Loss: 0.7458 | Test Acc: 0.5752
Training Complete
